In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━ 3.1/5.2 MB 56.0 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import re
from tqdm import tqdm

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize
# from razdel import tokenize

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## load functions

In [ ]:
#@title
def clean_text_old(newtext):
  newtext = str(newtext)
  stopwords = [
  "'type': 'bold'"
  ,"'type': 'italic'"
  , "'type': 'mention', 'text': '@lentadnya'"
  ,"'href': " #убрать весь адрес http
  ,"'type': 'text_link'"
  ,"'text':"
  ,"{" ,"}"
  ,', ,'
  , "[", "]"
  , "'"
  ]

  for stopword in stopwords:
    newtext = newtext.replace(stopword, "")

  replace_spaces =["\\n", "  ", ", ", "\\xa0"]
  for replace_space in replace_spaces:
    newtext = newtext.replace(replace_space, " ")

  # лишние пробелы похоже не влияют на предсказания модели. Зато эмоджи влияют. Оставляем только текстовые знаки и препинания.
  newtext = re.sub('[^А-Яа-яЁёA-Za-z0-9 _.,!-—?"«»]*', "", newtext)

  newtext = newtext.replace('ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'\
                            , '')

  return newtext

In [ ]:
morph = pymorphy2.MorphAnalyzer()
ALLOWED_POS_TAGS = {'NOUN', 'ADJF', 'ADJS', 'VERB', 'INFN', 'PRTF', 'GRND', 'PRTS', 'ADVB'}

def clean_text(newtext):
  newtext = str(newtext)

  newtext = newtext.replace('ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'\
                            , '')

  current_tokens = simple_word_tokenize(newtext)
  current_pos_filtered_lemmata = []
  for token in current_tokens:
      parsed = morph.parse(token)[0]
      if parsed.tag.POS in ALLOWED_POS_TAGS:
          current_pos_filtered_lemmata.append(parsed.normal_form)

  newtext = ' '.join(current_pos_filtered_lemmata)

  return newtext

In [ ]:
def filter_war_texts(df_test):
  df_war_news = df_test[
        (df_test['text'].str.contains('воен|войн|спецоперац|обстрел|арм|СВО', case=False)) 
        & ~(df_test['text'].str.contains('Главное к утру|Главные события|Главные новости', case=True)) #убираем сводки/дайджесты из нескольких новостей
        & (df_test['text'].str.contains('укр|ВСУ|Азов', case=False))
        ]
  return df_war_news

def read_clean_json_news(file_path, date_from='2022-01-01'):
  # read json-file to dataframe
  df_js = pd.read_json(file_path)
  df_full = pd.DataFrame(df_js['messages'].tolist())
  df_full = df_full[['id', 'date', 'text']]
  df_full['date'] = pd.to_datetime(df_full['date'])
  # select specific dates
  df_test = df_full[df_full['date'] > date_from]
  # filter news by key words
  df_test = filter_war_texts(df_test)
  # clean news texts and put into 'news' column
  df_test['news'] = df_test.apply(lambda row: clean_text(row['text']), axis=1)
  df_test.drop('text', axis=1, inplace=True)
  # clean of empty news
  df_test = df_test[df_test['news'].str.len()>5]
  return df_test

In [ ]:
df_js = pd.read_json(path)  
df_full = pd.DataFrame(df_js['messages'].tolist())
df_full = df_full[['id', 'date', 'text']]
df_full['date'] = pd.to_datetime(df_full['date'])
# select specific dates
df_test = df_full[df_full['date'] > date_from]


In [ ]:
def filter_war_news(df_test):
  df_war_news = df_test[
        (df_test['news'].str.contains('воен|войн|спецоперац|обстрел|арм|СВО', case=False)) 
        & ~(df_test['news'].str.contains('Главное к утру|Главные события|Главные новости', case=True)) #убираем сводки/дайджесты из нескольких новостей
        & (df_test['news'].str.contains('укр|ВСУ|Азов', case=False))
        ]
  return df_war_news

## get texts ready

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1) NEWS LOAD & PRER
# PARAMETERS
paths = [
        '/content/drive/MyDrive/Colab Notebooks/Datasets/news TG channels/meduza - from jan22.json',
        '/content/drive/MyDrive/Colab Notebooks/Datasets/news TG channels/rt.com - from jan22.json',
        '/content/drive/MyDrive/Colab Notebooks/Datasets/news TG channels/bbc - from jan22.json',
        '/content/drive/MyDrive/Colab Notebooks/Datasets/news TG channels/ria - from jan22.json'
        ]
date_from = '2022-02-24'

df_news_list = []
for path in paths:
  df_news_ = read_clean_json_news(path, date_from)
  df_news_['source'] = path[65:68]
  df_news_list.append(df_news_)

df_news = pd.concat(df_news_list, axis=0)

df_war_news = filter_war_news(df_news) # run this to filter by key words on war & remove digests

TypeError: ignored

In [ ]:
len(df_war_news)

3122

In [ ]:
df_war_news.head(5)

In [ ]:
df_war_news.reset_index(inplace=True)

In [ ]:
df_war_news['news'][0]

NameError: ignored

In [ ]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

text = df_war_news['news'][0]

tokens = mystem.lemmatize(text.lower())
tokens = [token for token in tokens if token not in russian_stopwords\
              and token != ' ' \
              and token.strip() not in punctuation]
    
text = ' '.join(tokens)
    
text

'неизвестный дважды пересекать укреплять граница северный южный корея ноябрь сбегать кндр январь  —  вернуться назад суббота южнокорейский военный замечать мужчина демилитаризованный зона который разделять корейский полуостров поздно выясняться граница нарушать перебежчик кндр который ноябрь бежать северный корея южный заявлять минобороны южный корея основание считать мужчина шпион дальнейший судьба перебежчик неизвестный однако кндр действовать крайне жесткий антиковидный мера военный мочь стрелять поражение допускать попадание вирус страна type link https :// mdza io nnrx3gokumw'

## BERTopic

In [ ]:
# Topic model
from bertopic import BERTopic
# Dimension reduction
from umap import UMAP

In [ ]:
# 13K mews = 26min
# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="russian", calculate_probabilities=True)
# Run BERTopic model
topics, probabilities = topic_model.fit_transform(df_war_news['news'])

In [ ]:
# topics
df_war_news['class'] = topics

In [ ]:
df_war_news.to_excel('/content/drive/MyDrive/Colab Notebooks/Datasets/news TG channels/results/df_war_news_w_topics.xls')

In [ ]:
# Topics by sources
pd.crosstab(df_war_news['class'], df_war_news['source']
            , margins=True
            , normalize='columns'
            ).sort_values('All', ascending=False)#.to_excel('/content/drive/MyDrive/Colab Notebooks/Datasets/news TG channels/results/sources_by_topics_percent.xls')

pd.crosstab(df_war_news['class'], df_war_news['source']
            , margins=True
            # , normalize='columns'
            ).sort_values('All', ascending=False).to_excel('/content/drive/MyDrive/Colab Notebooks/Datasets/news TG channels/results/sources_by_topics_abs.xls')

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,6162,-1_не_что_на_по
1,0,302,0_нас_они_мы_всу
2,1,239,1_переговоров_переговоры_переговорах_песков
3,2,210,2_ск_дело_уголовное_поручил
4,3,182,3_кадры_показало_минобороны_ка
...,...,...,...
181,180,10,180_d7_хасиды_умани_нахмана
182,181,10,181_прокопенко_командир_азовстали_волына
183,182,10,182_алёша_выбегает_завирусился_восьмилетний
184,183,10,183_одессе_попала_одесской_юг


In [ ]:
len(set(topics))

186

In [ ]:
# Get top 10 terms for a topic
topic_model.get_topic(35)

[('погибли', 0.039844494815486846),
 ('результате', 0.03576036521639366),
 ('ранены', 0.030219214437031103),
 ('человек', 0.028002647286030916),
 ('обстрела', 0.025951746863622942),
 ('обстрелов', 0.025440721036078792),
 ('d0', 0.023260355506026807),
 ('пострадали', 0.018289239296785692),
 ('днр', 0.017818080724079283),
 ('мирных', 0.017777894008617384)]

In [ ]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=50)